In [1]:
!pip install gensim

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
!cd gdrive/My' 'Drive/Colab' 'Notebooks
!ls gdrive/My' 'Drive/Colab' 'Notebooks
import os
os.chdir("gdrive/My Drive/Colab Notebooks")

 bertmodel		        modelvc_colab.model.trainables.syn1neg.npy
 bert_trained.png	        modelvc_colab.model.wv.vectors.npy
 bot_model3.py		        modelvc.model
 bot_model_fin2.py	        modelvc.model.syn1neg.npy
 bot_model_final.py	        modelvc.model.wv.vectors.npy
 bot_model_fin.py	        movie_conversations.txt
 bot_models.py		        movie_lines.txt
 bot_train_savefile.py	        pre2_run.ipynb
 conv.txt		        pre2_savefile.ipynb
'Copy of pre2_savefile.ipynb'   pre.ipynb
'Copy of pre.ipynb'	        __pycache__
 data_2.json		        questions-word2s.txt
 datapre.py		        questions-words.txt
 embeddings_colab.kv	        td.txt
 embeddings.kv		        test.txt
 fc_net_git.py		        Untitled
 fc_net.py		        Untitled1.ipynb
 layers			        Untitled2.ipynb
 model			        Untitled3.ipynb
 modelvc_colab.model	        w2v.txt


In [4]:
from datapre import word2vec
import numpy as np
import json

settings = {
    'window_size': 2,
    'n': 200,
    'epochs': 100,
    'learning_rate':0.1,
    'randepochs':0
}
dataname = "data_2.json"
max_seq_len = 10
limit = 10000000
def plot():
    from numpy import linalg as LA

    with open(dataname) as json_file:
        data = json.load(json_file)

    words = list(data.keys())
    v_count = len(words)
    n = settings['n']
    input = np.zeros((v_count, n))

    dim = n
    input = np.zeros((v_count, n))
    for i in range(v_count):
        word = words[i]
        input[i, :] = data[word]
    input = input.T
    Cov = 1 / dim * input.dot(input.T)
    w, v2 = LA.eig(Cov)
    wid = np.argsort(w)
    wid_1 = wid[dim - 1]
    wid_2 = wid[dim - 2]
    v = np.zeros((dim, 2))
    v[:, 0] = v2[:, wid_1]
    v[:, 1] = v2[:, wid_2]
    output = input
    output = v.T.dot(input)
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.scatter(output[0,:],output[1,:])
    #maxx = np.max(output[0,:])
    #maxy = np.max(output[1, :])
    #minx = np.min(output[0, :])
    #miny = np.min(output[1, :])
    for i in range(v_count):
      ax.annotate(words[i],(output[0,i],output[1,i]))
    ax.axis('auto')
    ax.set_autoscale_on(True)
    #ax.plot([minx,maxx],[miny,maxy])
    plt.show()

def movieinput():
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    import re
    import time
    lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
    conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    convs = []
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
        convs.append(_line.split(','))
    questions = []
    answers = []

    for conv in convs:
        for i in range(len(conv) - 1):
            questions.append(id2line[conv[i]])
            answers.append(id2line[conv[i + 1]])
    corpust = questions + answers
    corpus = [re.sub(r'[^A-Za-z0-9 -]+', '', word.lower()) + ' <EOS>' for word in corpust]
    return corpus

In [5]:
corpus = movieinput()

In [6]:
corpus2 = [word.split() for word in corpus]
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(corpus2, min_count = 1, size=200, window=5,  workers=4)

In [7]:
with open("/content/gdrive/MyDrive/Colab Notebooks/embeddings_colab.kv", "wb") as handle:
    model.wv.save(handle)
model.save("/content/gdrive/MyDrive/Colab Notebooks/modelvc_colab.model")